In [1]:
# !pip install python==3.7
!pip install panda3d==1.10.5
!pip install cython==0.29.21
!pip install gym==0.17.3
!pip install pgdrive==0.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 MB 5.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.0 MB/s eta 0:00:0000:0100:01
  Using cached gym-0.17.3.tar.gz (1.6 MB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 24.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 31.7 MB/s eta 0:00:0000:0100:01
  Using cached pyglet-1.5.0-py2.py3-none-any.whl (1.0 MB)
  Using cached cloudpickle-1.6.0-py3-none-any.whl (23 kB)
  Using cached future-0.18.3.tar.gz (840 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654622 sha256=6fc2b9f87afb029a2160c31f4ef720470e831465233eaad89a317fe1e61bf685
  Stored in directory: /home/rickbook/.cache/pip/wheels/d1/81/4b/dd9c029691022cb957398d1f015e66b75e37637dda61abdf58
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492022 sha256=ed7a

In [4]:
!pip3 install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 5.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 14.3 MB/s eta 0:00:0000:0100:01
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)


In [10]:
from pgdrive import PGDriveEnv
from pgdrive.examples import expert
import gym

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from pgdrive.envs import PGDriveEnv

import math

Python 3.11.0


In [1]:
class DriveAgent(nn.Module):
    def __init__(self, obs_space, action_space):
        super(DriveAgent, self).__init__()
        # self.obs_space = obs_space
        # self.action_space = action_space
        self.mlp = nn.Sequential(
            nn.Linear(275, 16),
            nn.GELU(),
            nn.LayerNorm(16),
            nn.Dropout(0.2),
            nn.Linear(16, 8),
            nn.GELU(),
            nn.LayerNorm(8),
            nn.Dropout(0.2),
            nn.Linear(8, 2)
        )


    def forward(self, obs):
        x = self.mlp(obs)
        return x


# dont render the env
env = gym.make("PGDrive-v0", config=dict(use_render=False))  
# render the env
# env = gym.make("PGDrive-v0", config=dict(use_render=True))  

print("\nThe action space: {}".format(env.action_space))
print("\nThe observation space: {}\n".format(env.observation_space))
print("Starting the environment ...\n")



def train_agent(num_episodes = 50, lr = 5e-2, gamma = 0.1):

    agent = DriveAgent(env.observation_space, env.action_space)
    optimizer = optim.AdamW(agent.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=gamma)

    print(agent)

    for i in range(num_episodes):
        ep_reward = 0.0
        obs = env.reset()

        for i in range(num_episodes):
            
            logits = agent(torch.from_numpy(obs).float())

            obs , reward, done, info = env.step(logits.detach().numpy())

            target = torch.from_numpy(expert(obs)).float()

            ep_reward += reward

            optimizer.zero_grad()
            # loss 4
            loss = torch.nn.functional.mse_loss(logits, target) / (1/(math.pi**(reward))) # * int(info['out_of_road'])  # 'arrive_dest': True
            # print(loss)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # render the environment
            # env.render()
            if int(info['out_of_road']) == 1:
                env.reset()
                print("\nEpisode reward: ", ep_reward)
                break
            if int(info['arrive_dest']):
                env.reset()
                print("\nEpisode reward: ", ep_reward)
                break

        print("\nThe returned loss: {}.".format(loss))
        # obs.close()

        print("\nThe returned reward: {}.".format(reward))
        print("\nThe returned information: {}".format(info))

        print("\nPGDrive successfully run!")

    return agent



agent = train_agent(num_episodes=50, lr=5e-2, gamma=0.1)


# save the agent

torch.save(agent.state_dict(), "drive_agent_layer_16-8_50_loss1.pth")

Successfully registered the following environments: ['PGDrive-test-v0', 'PGDrive-validation-v0', 'PGDrive-v0', 'PGDrive-10envs-v0', 'PGDrive-1000envs-v0', 'PGDrive-training0-v0', 'PGDrive-training1-v0', 'PGDrive-training2-v0'].

The action space: Box(-1.0, 1.0, (2,), float32)

The observation space: Box(-0.0, 1.0, (275,), float32)

Starting the environment ...

DriveAgent(
  (mlp): Sequential(
    (0): Linear(in_features=275, out_features=16, bias=True)
    (1): GELU(approximate='none')
    (2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=16, out_features=8, bias=True)
    (5): GELU(approximate='none')
    (6): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=8, out_features=2, bias=True)
  )
)


:device(error): Error adding inotify watch on /dev/input: No such file or directory
:device(error): Error opening directory /dev/input: No such file or directory



Episode reward:  1.76317587918967

The returned loss: 309.1747741699219.

The returned reward: -5.

The returned information: {'cost': 0.0, 'velocity': 24.04790840148926, 'steering': -0.4812576472759247, 'acceleration': 1.0, 'step_reward': 0.0, 'crash': False, 'out_of_road': True, 'arrive_dest': False}

PGDrive successfully run!

Episode reward:  -5.0

The returned loss: 306.09515380859375.

The returned reward: -5.

The returned information: {'cost': 0.0, 'velocity': 3.531600022315979, 'steering': -0.5138373374938965, 'acceleration': 1.0, 'step_reward': 0.0, 'crash': False, 'out_of_road': True, 'arrive_dest': False}

PGDrive successfully run!

Episode reward:  0.9092953627303899

The returned loss: 309.2204284667969.

The returned reward: -5.

The returned information: {'cost': 0.0, 'velocity': 24.277472877502444, 'steering': -0.16951484978199005, 'acceleration': 1.0, 'step_reward': 0.0, 'crash': False, 'out_of_road': True, 'arrive_dest': False}

PGDrive successfully run!

Episode re

In [6]:
from pgdrive import PGDriveEnv
from pgdrive.examples import expert
import torch
import torch.nn as nn
import gym


class DriveAgent(nn.Module):
    def __init__(self, obs_space, action_space):
        super(DriveAgent, self).__init__()
        # self.obs_space = obs_space
        # self.action_space = action_space
        self.fc1 = nn.Linear(275, 8)
        self.fc2 = nn.Linear(8, 2)
        self.gelu = nn.GELU()
        self.dropuot = nn.Dropout(0.2)
        self.layer_norm = nn.LayerNorm(8)


    def forward(self, obs):
        x = self.gelu(self.fc1(obs))
        x = self.layer_norm(self.dropuot(x) + x) # Add & Norm
        x = self.fc2(x)
        return x
    
    
env = gym.make("PGDrive-10envs-v0", config=dict(use_render=True))  

# load the agent
agent = DriveAgent(env.observation_space, env.action_space)
agent.load_state_dict(torch.load("drive_agent_8_100_reverse_reward_double.pth"))

print("\nThe action space: {}".format(env.action_space))
print("\nThe observation space: {}\n".format(env.observation_space))
print("Starting the environment ...\n")

ep_reward = 0.0
obs = env.reset()
for i in range(1000):
    logits = agent(torch.from_numpy(obs).float())

    obs, reward, done, info = env.step(logits.detach().numpy())
    ep_reward += reward

    env.render()
    
    if info['out_of_road']:
        print("\nEpisode reward: ", ep_reward)
        break

print("\nThe returned reward: {}.".format(reward))
print("\nThe returned information: {}".format(info))

env.close()
print("\nPGDrive successfully run!")



The action space: Box(-1.0, 1.0, (2,), float32)

The observation space: Box(-0.0, 1.0, (275,), float32)

Starting the environment ...



Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
Xlib:  extension "XFree86-DGA" missing on display ":0".
:task(warning): Creating implicit AsyncTaskChain default for AsyncTaskManager TaskManager



Episode reward:  129.50468629473062

The returned reward: -5.

The returned information: {'cost': 0.0, 'velocity': 79.41844253540039, 'steering': 0.01143912598490715, 'acceleration': 0.5232788324356079, 'step_reward': 0.0, 'crash': False, 'out_of_road': True, 'arrive_dest': False}

PGDrive successfully run!


In [ ]:
env.reset()
env.close()

In [ ]:
# train the agent
from pgdrive import PGDriveEnv
from pgdrive.examples import expert
import gym

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from pgdrive.envs import PGDriveEnv

import math

####################################################################
### GLOBAL VARIABLES
CROSSOVER_MAGNITUDE = 0.1
MUTATION_FACTOR = 0.2
####################################################################

def attention_map_crossover(attention_map):
    """ Apply the crossover over the attention maps of each head.
    The crosseover consists in picking a random index in the maxtrix over
    the columns and swapping the values in between the columns of the
    attention map.
    
    Args:
        attention_map (torch.Tensor): shape (batch_size, number_of_heads, activation_size, activation_size)
        
    Returns:
        torch.Tensor: shape (batch_size, number_of_heads, activation_size, activation_size)
    
    """
    
    # get the crossover magnitude
    crossover_magnitude = CROSSOVER_MAGNITUDE
    
    # get the batch size
    dim_batch = attention_map.shape[0]
    
    # get the number of heads
    number_of_heads = attention_map.shape[1]
    
    for idx_batch in range(dim_batch):
        for idx_head in range(number_of_heads):
            
            # get the crossover index
            crossover_index = attention_map.shape[2] - int(attention_map.shape[2]*crossover_magnitude)
            
            # get two random indexes
            random_index_1 = torch.randint(0, attention_map.shape[2],(1,))[0]
            random_index_2 = torch.randint(0, attention_map.shape[2],(1,))[0]
            
            # swap the values in that position over the columns
            for idx, (x_1, x_2) in enumerate(zip(attention_map[idx_batch][idx_head][random_index_1][crossover_index:].detach(), attention_map[idx_batch][idx_head][random_index_2][crossover_index:].detach())):
                
                # debug
                # print(attention_map[idx_batch][idx_head].shape, random_index_1, random_index_2, crossover_index)         
                # print(x_1, x_2,idx_batch, idx_head, idx)
                
                # swap the values in that position over the columns
                attention_map[idx_batch][idx_head][random_index_1][crossover_index+idx] = x_2 # make crossover
                attention_map[idx_batch][idx_head][random_index_2][crossover_index+idx] = x_1 # make crossover
    
    return attention_map

################################################################

def mutate_attention_map(attention_map):
    """ Mutate the attention map by making an elementwise multiplication with 
    a random tensor with values between 1-mutation_factor and 1+mutation_factor
    
    Args:
        attention_map (torch.Tensor): shape (batch_size, num_heads, activation_size, activation_size)
        
    Returns:    
        torch.Tensor: shape (batch_size, num_heads, activation_size, activation_size)
    
    
    """
    # get the mutation factor
    mutation_factor = MUTATION_FACTOR
    # return the mutated attention map
    # multiplied elementwise with a 
    # random matrix with values between 
    # 1-mutation_factor and 1+mutation_factor
    return torch.mul(attention_map, torch.randn(attention_map.shape).uniform_(1-mutation_factor,1+mutation_factor).to(attention_map.device))
    
    
################################################################


def head_batched_attention_mechanism(Q, K, V):
    """
    Args:
        Q: (batch_size, num_heads, num_layer, activation_size)
        K: (batch_size, num_heads, num_layer, activation_size)
        V: (batch_size, num_heads, activation_size, 1) # activations in the current layer

    Returns:
        attention: (batch_size, num_heads, activation_size)

        # attention mechanism
        # # (batch_size, num_heads, activation_size, activation_size)
        # attention = torch.matmul(Q, K.transpose(-1,-2))
        # attention = attention / torch.sqrt(torch.tensor(activation_size).float())

        # # (batch_size, num_heads, activation_size, 1)
        # attention = nn.Softmax(dim=-1)(attention)

        # # (batch_size, num_heads, activation_size, 1)
        # attention = torch.matmul(attention, V)

        # # (batch_size, num_heads, activation_size)
        # attention = attention.squeeze(-1)

    """
    
    # with probability p
    p = torch.rand(1)
    
    # p <= 0.6 apply the mutation only
    if p <= 0.6:
        return (nn.Softmax(dim=-1)(
                    mutate_attention_map(torch.matmul( 
                                Q , 
                                K.transpose(-1,-2)
                    )/torch.sqrt(torch.tensor(8)))    
                ) @ V).squeeze(-1)
        
    # p > 0.6 apply the crossover only
    else:
        return (nn.Softmax(dim=-1)(
                attention_map_crossover(torch.matmul( 
                            Q , 
                            K.transpose(-1,-2)
                )/torch.sqrt(torch.tensor(8)))    
            ) @ V).squeeze(-1)



class LinW_Attention_Module_C_M(nn.Module):
    def __init__(self, dim_emb, n_head, out_dim = None) -> None:
        super(LinW_Attention_Module_C_M, self).__init__()

        assert dim_emb % n_head == 0, 'dim_emb must be divisible by n_head'

        self.dim_emb = dim_emb
        self.n_head = n_head

        if out_dim is None:
            self.W_O = nn.Linear(dim_emb*n_head, dim_emb)
        else:
            self.W_O = nn.Linear(dim_emb*n_head, out_dim)

    def forward(self, x):
        # get the input
        Q, K, V = x.unsqueeze(0), x.unsqueeze(0), x.unsqueeze(0)
        # get the shape of the input
        batch_size, activation_size = Q.size()
        
        # # check the device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # move to device
        Q.to(device)
        K.to(device)
        V.to(device)
        
        # reshape Q, K, V
        # parallelize over the number of heads
        # (batch_size, num_heads, num_layer, activation_size)
        Q = torch.stack([Q for _ in range(self.n_head)], 1)
        K = torch.stack([K for _ in range(self.n_head)], 1)
        V = torch.stack([V for _ in range(self.n_head)], 1)

        Q = Q.reshape(batch_size, self.n_head, activation_size, 1)
        K = K.reshape(batch_size, self.n_head, activation_size, 1)
        V = V.reshape(batch_size, self.n_head, activation_size, 1)
        
        # apply attention mechanism
        out_attention = head_batched_attention_mechanism(Q, K, V).reshape(batch_size, self.n_head*self.dim_emb)

        # apply linear transformation
        return self.W_O(out_attention).squeeze(0)
    

####################################################################


def get_activations_per_object(activations):
    """ Get the activations for each object per layer

    Args:
        activations (torch.Tensor): shape (num_layers, batch_size, number_activations)

    Returns:
        torch.Tensor: shape (nr_object, num_layers, activation_for_each_layer)

    """
    return torch.stack([activations[:,i,:] for i in range(activations.shape[1])])


####################################################################


def get_layer_activations(activations):
    """ Get the activations for each layer for each sample

    Args:
        activations (torch.Tensor): shape (batch_size, number_activations)
        batch_size (int): batch size
        number_activations (int): number of activations

    Returns:
        torch.Tensor: shape (nr_object, num_layers, activation_for_each_layer)

    """
    return get_activations_per_object(torch.stack(activations))


####################################################################


def get_activations_per_object(activations):
    """ Get the activations for each object per layer

    Args:
        activations (torch.Tensor): shape (num_layers, batch_size, number_activations)

    Returns:
        torch.Tensor: shape (nr_object, num_layers, activation_for_each_layer)

    """
    return torch.stack([activations[:,i,:] for i in range(activations.shape[1])])


####################################################################


def get_layer_activations(activations):
    """ Get the activations for each layer for each sample

    Args:
        activations (torch.Tensor): shape (batch_size, number_activations)
        batch_size (int): batch size
        number_activations (int): number of activations

    Returns:
        torch.Tensor: shape (nr_object, num_layers, activation_for_each_layer)

    """
    return get_activations_per_object(torch.stack(activations))


####################################################################



class DriveAgent(nn.Module):
    def __init__(self,):
        super(DriveAgent, self).__init__()
        self.fc1 = nn.Linear(275, 8)
        self.linW = LinW_Attention_Module_C_M(8, 2)
        self.fc2 = nn.Linear(8, 2)
        self.gelu = nn.GELU()
        self.dropuot = nn.Dropout(0.2)
        self.layer_norm = nn.LayerNorm(8)


    def forward(self, obs):
        x = self.gelu(self.fc1(obs))
        x = self.layer_norm(x)
        x = self.linW(x)
        x = self.gelu(x)
        x = self.fc2(x)
        return x



# dont render the env
env = gym.make("PGDrive-v0", config=dict(use_render=False))  

# render the env
# env = gym.make("PGDrive-v0", config=dict(use_render=True))  

print("\nThe action space: {}".format(env.action_space))
print("\nThe observation space: {}\n".format(env.observation_space))
print("Starting the environment ...\n")


def train_agent(num_episodes = 50, lr = 5e-2, gamma = 0.1):
    # create the agent
    agent = DriveAgent()
    # create the optimizer
    optimizer = optim.AdamW(agent.parameters(), lr=lr)
    # create the scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2, eta_min=0.0001, last_epoch=-1)

    print(agent)

    for i in range(num_episodes):
        ep_reward = 0.0
        obs = env.reset()

        for i in range(num_episodes):
            
            logits = agent(torch.from_numpy(obs).float())

            obs , reward, done, info = env.step(logits.detach().numpy())

            target = torch.from_numpy(expert(obs)).float()

            ep_reward += reward

            optimizer.zero_grad()
            # loss = torch.nn.functional.mse_loss(logits, target) / (1/(math.pi**(ep_reward))) 
            if reward <= 0:
                loss = torch.nn.functional.mse_loss(logits, target) * abs(ep_reward)
            else:
                loss = 50/abs(reward) + torch.nn.functional.mse_loss(logits, target) * abs(ep_reward)
            # print(loss)
            loss.backward()
            optimizer.step()
            scheduler.step()

            # render the environment
            # env.render()
            if i > 100 and int(info['out_of_road']) == 1:
                env.reset()
                print("\nEpisode reward: ", ep_reward)
                break
            if int(info['arrive_dest']):
                env.reset()
                print("\nEpisode reward: ", ep_reward)
                break

        print("\nThe returned loss: {}.".format(loss))
        # obs.close()

        print("\nThe returned reward: {}.".format(reward))
        print("\nThe returned information: {}".format(info))

        print("\nPGDrive successfully run!")

    return agent


agent = train_agent(num_episodes=100, lr=5e-2, gamma=0.1)


# save the agent
torch.save(agent.state_dict(), "pseudo_attention_agent.pth")

In [ ]:
# render the animation of the experiment
from pgdrive import PGDriveEnv
from pgdrive.examples import expert
import gym

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from pgdrive.envs import PGDriveEnv

import math

####################################################################
### GLOBAL VARIABLES
CROSSOVER_MAGNITUDE = 0.1
MUTATION_FACTOR = 0.2
####################################################################

def attention_map_crossover(attention_map):
    """ Apply the crossover over the attention maps of each head.
    The crosseover consists in picking a random index in the maxtrix over
    the columns and swapping the values in between the columns of the
    attention map.
    
    Args:
        attention_map (torch.Tensor): shape (batch_size, number_of_heads, activation_size, activation_size)
        
    Returns:
        torch.Tensor: shape (batch_size, number_of_heads, activation_size, activation_size)
    
    """
    
    # get the crossover magnitude
    crossover_magnitude = CROSSOVER_MAGNITUDE
    
    # get the batch size
    dim_batch = attention_map.shape[0]
    
    # get the number of heads
    number_of_heads = attention_map.shape[1]
    
    for idx_batch in range(dim_batch):
        for idx_head in range(number_of_heads):
            
            # get the crossover index
            crossover_index = attention_map.shape[2] - int(attention_map.shape[2]*crossover_magnitude)
            
            # get two random indexes
            random_index_1 = torch.randint(0, attention_map.shape[2],(1,))[0]
            random_index_2 = torch.randint(0, attention_map.shape[2],(1,))[0]
            
            # swap the values in that position over the columns
            for idx, (x_1, x_2) in enumerate(zip(attention_map[idx_batch][idx_head][random_index_1][crossover_index:].detach(), attention_map[idx_batch][idx_head][random_index_2][crossover_index:].detach())):
                
                # debug
                # print(attention_map[idx_batch][idx_head].shape, random_index_1, random_index_2, crossover_index)         
                # print(x_1, x_2,idx_batch, idx_head, idx)
                
                # swap the values in that position over the columns
                attention_map[idx_batch][idx_head][random_index_1][crossover_index+idx] = x_2 # make crossover
                attention_map[idx_batch][idx_head][random_index_2][crossover_index+idx] = x_1 # make crossover
    
    return attention_map

################################################################

def mutate_attention_map(attention_map):
    """ Mutate the attention map by making an elementwise multiplication with 
    a random tensor with values between 1-mutation_factor and 1+mutation_factor
    
    Args:
        attention_map (torch.Tensor): shape (batch_size, num_heads, activation_size, activation_size)
        
    Returns:    
        torch.Tensor: shape (batch_size, num_heads, activation_size, activation_size)
    
    
    """
    # get the mutation factor
    mutation_factor = MUTATION_FACTOR
    # return the mutated attention map
    # multiplied elementwise with a 
    # random matrix with values between 
    # 1-mutation_factor and 1+mutation_factor
    return torch.mul(attention_map, torch.randn(attention_map.shape).uniform_(1-mutation_factor,1+mutation_factor).to(attention_map.device))
    
    
################################################################


def head_batched_attention_mechanism(Q, K, V):
    """
    Args:
        Q: (batch_size, num_heads, num_layer, activation_size)
        K: (batch_size, num_heads, num_layer, activation_size)
        V: (batch_size, num_heads, activation_size, 1) # activations in the current layer

    Returns:
        attention: (batch_size, num_heads, activation_size)

        # attention mechanism
        # # (batch_size, num_heads, activation_size, activation_size)
        # attention = torch.matmul(Q, K.transpose(-1,-2))
        # attention = attention / torch.sqrt(torch.tensor(activation_size).float())

        # # (batch_size, num_heads, activation_size, 1)
        # attention = nn.Softmax(dim=-1)(attention)

        # # (batch_size, num_heads, activation_size, 1)
        # attention = torch.matmul(attention, V)

        # # (batch_size, num_heads, activation_size)
        # attention = attention.squeeze(-1)

    """
    
    # with probability p
    p = torch.rand(1)
    
    # p <= 0.6 apply the mutation only
    if p <= 0.6:
        return (nn.Softmax(dim=-1)(
                    mutate_attention_map(torch.matmul( 
                                Q , 
                                K.transpose(-1,-2)
                    )/torch.sqrt(torch.tensor(8)))    
                ) @ V).squeeze(-1)
        
    # p > 0.6 apply the crossover only
    else:
        return (nn.Softmax(dim=-1)(
                attention_map_crossover(torch.matmul( 
                            Q , 
                            K.transpose(-1,-2)
                )/torch.sqrt(torch.tensor(8)))    
            ) @ V).squeeze(-1)



class LinW_Attention_Module_C_M(nn.Module):
    def __init__(self, dim_emb, n_head, out_dim = None) -> None:
        super(LinW_Attention_Module_C_M, self).__init__()

        assert dim_emb % n_head == 0, 'dim_emb must be divisible by n_head'

        self.dim_emb = dim_emb
        self.n_head = n_head

        if out_dim is None:
            self.W_O = nn.Linear(dim_emb*n_head, dim_emb)
        else:
            self.W_O = nn.Linear(dim_emb*n_head, out_dim)

    def forward(self, x):
        # get the input
        Q, K, V = x.unsqueeze(0), x.unsqueeze(0), x.unsqueeze(0)
        # get the shape of the input
        batch_size, activation_size = Q.size()
        
        # # check the device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # move to device
        Q.to(device)
        K.to(device)
        V.to(device)
        
        # reshape Q, K, V
        # parallelize over the number of heads
        # (batch_size, num_heads, num_layer, activation_size)
        Q = torch.stack([Q for _ in range(self.n_head)], 1)
        K = torch.stack([K for _ in range(self.n_head)], 1)
        V = torch.stack([V for _ in range(self.n_head)], 1)

        Q = Q.reshape(batch_size, self.n_head, activation_size, 1)
        K = K.reshape(batch_size, self.n_head, activation_size, 1)
        V = V.reshape(batch_size, self.n_head, activation_size, 1)
        
        # apply attention mechanism
        out_attention = head_batched_attention_mechanism(Q, K, V).reshape(batch_size, self.n_head*self.dim_emb)

        # apply linear transformation
        return self.W_O(out_attention).squeeze(0)
    

####################################################################


def get_activations_per_object(activations):
    """ Get the activations for each object per layer

    Args:
        activations (torch.Tensor): shape (num_layers, batch_size, number_activations)

    Returns:
        torch.Tensor: shape (nr_object, num_layers, activation_for_each_layer)

    """
    return torch.stack([activations[:,i,:] for i in range(activations.shape[1])])


####################################################################


def get_layer_activations(activations):
    """ Get the activations for each layer for each sample

    Args:
        activations (torch.Tensor): shape (batch_size, number_activations)
        batch_size (int): batch size
        number_activations (int): number of activations

    Returns:
        torch.Tensor: shape (nr_object, num_layers, activation_for_each_layer)

    """
    return get_activations_per_object(torch.stack(activations))


####################################################################


def get_activations_per_object(activations):
    """ Get the activations for each object per layer

    Args:
        activations (torch.Tensor): shape (num_layers, batch_size, number_activations)

    Returns:
        torch.Tensor: shape (nr_object, num_layers, activation_for_each_layer)

    """
    return torch.stack([activations[:,i,:] for i in range(activations.shape[1])])


####################################################################


def get_layer_activations(activations):
    """ Get the activations for each layer for each sample

    Args:
        activations (torch.Tensor): shape (batch_size, number_activations)
        batch_size (int): batch size
        number_activations (int): number of activations

    Returns:
        torch.Tensor: shape (nr_object, num_layers, activation_for_each_layer)

    """
    return get_activations_per_object(torch.stack(activations))


####################################################################



class DriveAgent(nn.Module):
    def __init__(self, obs_space, action_space):
        super(DriveAgent, self).__init__()
        # self.obs_space = obs_space
        # self.action_space = action_space
        self.fc1 = nn.Linear(275, 8)
        self.linW = LinW_Attention_Module_C_M(8, 2)
        self.fc2 = nn.Linear(8, 2)
        self.gelu = nn.GELU()
        self.dropuot = nn.Dropout(0.2)
        self.layer_norm = nn.LayerNorm(8)


    def forward(self, obs):
        x = self.gelu(self.fc1(obs))
        x = self.layer_norm(x)
        x = self.linW(x)
        x = self.gelu(x)
        x = self.fc2(x)
        return x


    
env = gym.make("PGDrive-10envs-v0", config=dict(use_render=True))  

# load the agent
agent = DriveAgent(env.observation_space, env.action_space)
agent.load_state_dict(torch.load("pseudo_attention_agent.pth"))

print("\nThe action space: {}".format(env.action_space))
print("\nThe observation space: {}\n".format(env.observation_space))
print("Starting the environment ...\n")

ep_reward = 0.0
obs = env.reset()
for i in range(1000):
    logits = agent(torch.from_numpy(obs).float())

    obs, reward, done, info = env.step(logits.detach().numpy())
    ep_reward += reward

    env.render()
    
    if info['out_of_road']:
        print("\nEpisode reward: ", ep_reward)
        break

print("\nThe returned reward: {}.".format(reward))
print("\nThe returned information: {}".format(info))

env.close()
print("\nPGDrive successfully run!")
